In [10]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!kaggle datasets download -d informateur234/sleep-health-and-lifestyle-dataset
!unzip sleep-health-and-lifestyle-dataset.zip
!rm sleep-health-and-lifestyle-dataset.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
unzip:  cannot find or open sleep-health-and-lifestyle-dataset.zip, sleep-health-and-lifestyle-dataset.zip.zip or sleep-health-and-lifestyle-dataset.zip.ZIP.
rm: cannot remove 'sleep-health-and-lifestyle-dataset.zip': No such file or directory


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
import pickle

In [13]:
# read dataset
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


Person ID: An identifier for each individual in the dataset.

Gender: The gender of the person (Male/Female).

Age: The age of the person in years.

Occupation: The occupation or profession of the person.

Sleep Duration (hours): The number of hours the person sleeps per day.

Quality of Sleep (scale: 1-10): A subjective rating of sleep quality (1 to 10).

Physical Activity Level (minutes/day): The daily minutes of physical activity.

Stress Level (scale: 1-10): A subjective rating of stress level (1 to 10).

BMI Category: The BMI category of the person (Underweight, Normal, Overweight).

Blood Pressure (systolic/diastolic): The blood pressure measurement (systolic over diastolic).

Heart Rate (bpm): Resting heart rate in beats per minute.

Daily Steps: The number of steps taken per day.

Sleep Disorder: The presence or absence of a sleep disorder (None, Insomnia, Sleep Apnea).

#Prepocessing

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [15]:
# mengisi data null dengan nilai "None" > tidak memiliki masalah ganggguan tidur
df.fillna('None', inplace=True)

In [16]:
df.isnull().sum()

Person ID                  0
Gender                     0
Age                        0
Occupation                 0
Sleep Duration             0
Quality of Sleep           0
Physical Activity Level    0
Stress Level               0
BMI Category               0
Blood Pressure             0
Heart Rate                 0
Daily Steps                0
Sleep Disorder             0
dtype: int64

In [17]:
df.describe()

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,187.500000,42.184492,7.132086,7.312834,59.171123,5.385027,70.165775,6816.844920
std,108.108742,8.673133,0.795657,1.196956,20.830804,1.774526,4.135676,1617.915679
min,1.000000,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000
25%,94.250000,35.250000,6.400000,6.000000,45.000000,4.000000,68.000000,5600.000000
50%,187.500000,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000
75%,280.750000,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000
max,374.000000,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000


In [18]:
df.nunique()

Person ID                  374
Gender                       2
Age                         31
Occupation                  11
Sleep Duration              27
Quality of Sleep             6
Physical Activity Level     16
Stress Level                 6
BMI Category                 4
Blood Pressure              25
Heart Rate                  19
Daily Steps                 20
Sleep Disorder               3
dtype: int64

In [19]:
# menghapus kolom id karena tidak terpakai
df.drop(labels = ['Person ID'], axis = 1, inplace = True)
df.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None
1,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None
3,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [20]:
# Menggunakan LabelEncoder untuk mengubah kolom kategorikal menjadi numerik
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Occupation'] = label_encoder.fit_transform(df['Occupation'])
df['BMI Category'] = label_encoder.fit_transform(df['BMI Category'])
df['Sleep Disorder'] = label_encoder.fit_transform(df['Sleep Disorder'])
df['Blood Pressure'] = label_encoder.fit_transform(df['Blood Pressure'])

In [21]:
df

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,27,9,6.1,6,42,6,3,11,77,4200,1
1,1,28,1,6.2,6,60,8,0,9,75,10000,1
2,1,28,1,6.2,6,60,8,0,9,75,10000,1
3,1,28,6,5.9,4,30,8,2,22,85,3000,2
4,1,28,6,5.9,4,30,8,2,22,85,3000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
369,0,59,5,8.1,9,75,3,3,23,68,7000,2
370,0,59,5,8.0,9,75,3,3,23,68,7000,2
371,0,59,5,8.1,9,75,3,3,23,68,7000,2
372,0,59,5,8.1,9,75,3,3,23,68,7000,2


#Modelling

In [22]:
# inisialisasi data
X = df.drop(columns=['Sleep Disorder'])
y = df['Sleep Disorder']

In [23]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
# Inisialisasi StandardScaler
scaler = StandardScaler()

# Latih scaler pada data latih dan transformasikan data latih
X_train_scaled = scaler.fit_transform(X_train)

# Terapkan scaler yang telah dilatih pada data uji
X_test_scaled = scaler.transform(X_test)


In [25]:
# Create a QDA model
qda = QuadraticDiscriminantAnalysis()

# Fit the model to your training data (X) and target variable (y)
qda.fit(X_train_scaled, y_train)

# Use the model to make predictions on new data (X_new)
y_pred = qda.predict(X_test_scaled)

# Mengevaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9203539823008849
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86        24
           1       0.94      0.98      0.96        62
           2       0.96      0.81      0.88        27

    accuracy                           0.92       113
   macro avg       0.91      0.89      0.90       113
weighted avg       0.92      0.92      0.92       113



In [26]:
# pickle.dump(qda, open('model.pkl', 'wb'))
# Save the model
with open('model.pkl', 'wb') as file:
    pickle.dump(qda, file)